In [2]:
library(dplyr)
library(tidyverse)
library(magrittr)
library(RColorBrewer)
library(ggrepel)
library(ggpubr)

In [3]:
in_vec <- function(refvec, vecx){
  out_vec <- numeric(length(vecx))
  for (x in c(1:length(vecx))){
    if (vecx[x] %in% refvec){
      out_vec[x] <- 1
    }
    else {
      out_vec[x] <- 0
    }
  }
  return(out_vec)
}

in_vec_name <- function(refvec, vecx){
  out_vec <- character(length(vecx))
  for (x in c(1:length(vecx))){
    if (vecx[x] %in% refvec){
      out_vec[x] <- vecx[x]
    }
    else {
      out_vec[x] <- ""
    }
  }
  return(out_vec)
}

In [12]:
wk.dir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/7_CRF_Screen'
setwd(wk.dir)

z.p.file <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/7_CRF_Screen/5_zscore_div_sqrt_pval/all_z-score_div_sqrt-p_sqrt.csv'
z.p.tb <- read_csv(z.p.file)

crf.file <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/7_CRF_Screen/CRF_complexes.csv'
crf.tb <- read_csv(crf.file)
crf.complex.vec <- unique(crf.tb$complex)


── Column specification ───────────────────────────────────────────
cols(
  gene_name = col_character(),
  Q4minusQ1 = col_double(),
  Q3minusOther = col_double(),
  InputMinusAvg = col_double()
)



── Column specification ───────────────────────────────────────────
cols(
  complex = col_character(),
  gene_name = col_character()
)




In [9]:
length(crf.complex.vec)

[1] 27

In [ ]:
use_cols <- c('Q4minusQ1', 'Q3minusOther','InputMinusAvg') 
#anno.vec <- c("Tbx21", "Id2", "Runx3", "Mbd2", "Suv39h1", "Rpa3", "Bop1", "Chd7", "Smarca4", "Smarcc2")
#name.root <- "Baf"

for (use_col in use_cols){
    n <- 1
    barplot.list <- list()
    for (complex.i in crf.complex.vec){
        name.root <- complex.i
        crf.genes <- crf.tb %>% filter(complex == complex.i) %>% .$gene_name
        anno.vec <- str_to_title(crf.genes)

        dir.create(file.path(wk.dir, name.root), showWarnings = FALSE)
        setwd(file.path(wk.dir, name.root))
    
        out.name <- paste(name.root, paste(use_col, "bar", sep="."), sep="_")

        z.p.tb.use <- z.p.tb %>% select(one_of(c('gene_name', use_col)))
        colnames(z.p.tb.use) <- c('gene_name', 'val')

        # Rank order
        z.p.tb.use <- z.p.tb.use %>% arrange(val)
        z.p.tb.use$gene_name <- factor(z.p.tb.use$gene_name, levels=z.p.tb.use$gene_name)

        # Set color for top and bottom quarter
        col_panel <- c( "deepskyblue", "snow2", "tomato")
        qt <- as.integer(floor(nrow(z.p.tb.use)/4))
        col.vec <- rep(col_panel[1], qt)
        col.vec <- c(col.vec, rep(col_panel[2], nrow(z.p.tb.use)-2*qt))
        col.vec <- c(col.vec, rep(col_panel[3], qt))
        z.p.tb.use$color_use <- col.vec

        # Select annotations
        z.p.tb.use <- z.p.tb.use %>% 
        mutate(pointsize = in_vec(anno.vec, as.character(z.p.tb.use$gene_name))) %>%
        mutate(annoname = in_vec_name(anno.vec, as.character(z.p.tb.use$gene_name)))

        # Plot
        bar.plot <- ggplot(z.p.tb.use, aes(gene_name, val, fill=col.vec)) +
            geom_col(alpha=0.7) +
            geom_point(size=z.p.tb.use$pointsize, stroke = 0) +
            scale_fill_manual(values=col_panel) +
            coord_flip() +
            scale_y_continuous(position = "right", limits=c(-5.2, 5.2)) +
            geom_hline(yintercept=0, size=0.25) +
            theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
                  panel.background = element_rect(fill = "white",colour = "white", size = 0.5, linetype = "solid"),
                  axis.line.x = element_line(colour="black", size=0.5), axis.title.x = element_blank(), 
                  axis.ticks.y = element_blank(), axis.text.y=element_blank(), axis.title.y = element_blank(),
                  legend.position = "none")

        ggsave(paste(out.name, "_nolabel.png", sep=""), bar.plot, width=6, height=9, units="cm")

        bar.plot <- bar.plot +
            geom_text_repel(aes(label=annoname), force=25, min.segment.length=0, max.overlaps = Inf)

        ggsave(paste(out.name, ".pdf", sep=""), bar.plot, width=6, height=9, units="cm")
        ggsave(paste(out.name, ".png", sep=""), bar.plot, width=6, height=9, units="cm")
        
        barplot.list[[complex.i]] <- bar.plot + ggtitle(complex.i) + theme(plot.title = element_text(hjust = 0.5))
        n <- n + 1        
    } 
    all.plots <- ggarrange(plotlist=barplot.list, ncol=6, nrow=5)
    all.name <- file.path(wk.dir, paste(use_col,'_all-CRF','.pdf', sep=""))
    ggsave(all.name, device='pdf', width = 50, height = 60, dpi = 300, units = "cm")
}

